# Processing COMTRADE flow data

This script attempts to process the COMTRADEflow data for energy, coal, oil and gas. The goal is to extract the import, export, and flow data as geospatial information: the tools for which are found in the infra_tools library

In [1]:
import sys, os, importlib

import pandas as pd
import geopandas as gpd
import numpy as np

from shapely.geometry import Point, LineString

sys.path.append('../')

from infrasap import process_flows

In [2]:
input_file_energy = "/home/wb411133/data/Projects/INFRA/FLOWS/UN_comtrade_energy_flows.csv"
input_file_coal = "/home/wb411133/data/Projects/INFRA/FLOWS/Coal Ex-Im_UN Comtreade_June 18.csv"
input_file_oil = "/home/wb411133/data/Projects/INFRA/FLOWS/Oil UNComtrade 0115.csv"
input_file_gas = "/home/wb411133/data/Projects/INFRA/FLOWS/Gas UNComtrade 0115.csv"
out_folder = ""
global_boundaries = "/home/wb411133/data/Projects/INFRA/FLOWS/national_centroids.shp"

inB = gpd.read_file(global_boundaries)
if inB.crs != {'init':'epsg:4326'}:
    inB = inB.to_crs({'init':'epsg:4326'})



In [3]:
xx = pd.read_csv(input_file_oil)
xx.head()

,Classification,Year,Period,Period Desc.,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter Code,Reporter,...,Commodity,Qty Unit Code,Qty Unit,Qty,Netweight (kg),Gross weight (kg),Trade Value (US$),CIF Trade Value (US$),FOB Trade Value (US$),Flag
0,S3,2010,2010,2010,3,0,1,Import,8,Albania,...,"PETROLEUM OILS, CRUDE",8,Weight in kilograms,21972122.0,21972122.0,NaN,13047443.0,NaN,NaN,0
1,S3,2010,2010,2010,3,0,2,Export,8,Albania,...,"PETROLEUM OILS, CRUDE",8,Weight in kilograms,537315248.0,537315248.0,NaN,162736020.0,NaN,NaN,0
2,S3,2010,2010,2010,3,0,2,Export,8,Albania,...,"PETROLEUM OILS, CRUDE",8,Weight in kilograms,3.0,3.0,NaN,6.0,NaN,NaN,0
3,S3,2010,2010,2010,3,0,1,Import,8,Albania,...,"PETROLEUM OILS, CRUDE",8,Weight in kilograms,8.0,8.0,NaN,48.0,NaN,NaN,0
4,S3,2010,2010,2010,3,0,2,Export,8,Albania,...,"PETROLEUM OILS, CRUDE",8,Weight in kilograms,326910458.0,326910458.0,NaN,97263384.0,NaN,NaN,0


In [4]:
xx.columns

Index(['Classification', 'Year', 'Period', 'Period Desc.', 'Aggregate Level',
       'Is Leaf Code', 'Trade Flow Code', 'Trade Flow', 'Reporter Code',
       'Reporter', 'Reporter ISO', 'Partner Code', 'Partner', 'Partner ISO',
       'Commodity Code', 'Commodity', 'Qty Unit Code', 'Qty Unit', 'Qty',
       'Netweight (kg)', 'Gross weight (kg)', 'Trade Value (US$)',
       'CIF Trade Value (US$)', 'FOB Trade Value (US$)', 'Flag'],
      dtype='object')

In [5]:
xx['Commodity Code'].value_counts()

334    106995
335     50483
333     16415
Name: Commodity Code, dtype: int64

In [6]:
xx['Commodity'].value_counts()

PETROLEUM PRODUCTS          106995
RESIDUAL PETROL.PRODUCTS     50483
PETROLEUM OILS, CRUDE        16415
Name: Commodity, dtype: int64

In [8]:
sel = xx.loc[(xx['Reporter ISO'] == "JPN") & (xx['Trade Flow'] == "Import") & (xx['Year'] == 2017.0)]
sel.groupby("Commodity Code").agg({'Qty':'sum', "Qty":'sum'})/1000000

,Qty
Commodity Code,
333,315972.943900
334,51100.128012


In [10]:
process_flows.comtrade_flow?

In [11]:
good_columns=['Qty Unit Code', 'Year', 'Trade Flow', 'Reporter ISO', 'Partner ISO', 'Commodity', 'Qty', 'Trade Value (US$)', 'Reporter_Pt', 'Partner_Pt']

In [16]:
importlib.reload(process_flows)
file_def = [input_file_oil, "Oil", [333]]
in_file = file_def[0]
commodity = file_def[1]
oil_flows = process_flows.comtrade_flow(in_file, commodity, good_columns=good_columns)
oil_flows.initialize([333], inB, val_fields=['Trade Value (US$)', 'Qty'])

/home/wb411133/.conda/envs/geog/lib/python3.7/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)
/home/wb411133/.conda/envs/geog/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [17]:
oil_flows.country_flows.head()

,Reporter ISO,Partner ISO,Year,Trade Flow,Trade Value (US$),Qty,Reporter_Pt,Partner_Pt,geometry


In [13]:
oil_flows.save_simple_layers("/home/wb411133/data/Projects/INFRA/FLOWS/OIL_CRUDE_ONLY_2021/GEOJSON", "GEOJSON")

AttributeError: 'comtrade_flow' object has no attribute 'country_flows'

In [ ]:
importlib.reload(process_flows)

for file_def in [
                #[input_file_energy, "Energy", []],
                #[input_file_coal, "Coal", []],
                [input_file_oil, "Oil", []],
                #[input_file_gas, "Gas", []]
                ]:
    print(file_def[1])
    in_file = file_def[0]
    commodity = file_def[1]
    coal_flows = process_flows.comtrade_flow(in_file, commodity)
    #coal_flows.initialize(file_def[2], inB)
    #coal_flows.save_simple_layers("/home/wb411133/data/Projects/INFRA/FLOWS/%s/GEOJSON" % commodity, "GEOJSON")

In [ ]:
coal_flows.initialize?

In [ ]:
importlib.reload(process_flows)
base_out_folder = "/home/wb411133/data/Projects/INFRA/FLOWS"
data_res = {}
for file_def in [
                 [input_file_energy, "Energy", [3]],
                 [input_file_coal, "Coal", [8]],
                 #[input_file_gas, "Gas", [8,1]],
                 #[input_file_oil, "Oil", [8]]
                ]:
    print("Processing %s" % file_def[1])
    data_flows = process_flows.comtrade_flow(file_def[0], file_def[1])
    data_flows.initialize(file_def[2], inB)
    select_folder = os.path.join(base_out_folder, file_def[1])
    for file_type in ["SHP"]:
        out_folder = os.path.join(select_folder, file_type)
        data_flows.save(out_folder, file_type)    
    data_res[file_def[1]] = data_flows

# Processing airport data from Heinrich

In [ ]:
airport_flows = "/home/wb411133/data/Projects/INFRA/FLOWS/Airport_Volume.csv"

inD = pd.read_csv(airport_flows)
#inD.drop(['Country Name', 'Country Name.1'], axis=1, inplace=True)

In [ ]:
inD.head()

In [ ]:
# extract airport locations
inD_grouped = inD.groupby(['Orig','Year'])
d = {'Name':'first','TotalSeats':'sum', "Country Name":"first","Airport1Latitude":'first', "Airport1Longitude":'first'}
airport_locations = inD_grouped.agg(d)
airport_locations.head()

In [ ]:
airport_locations = airport_locations.groupby(level=0).last()
airport_locations.head()

In [ ]:
airport_locations.to_csv(airport_flows.replace(".csv", "_airport_locations.csv"))

In [ ]:
#Calculate out_flows
d = {'TotalSeats':'sum'}
inD_grouped = inD.groupby(['Country1', "Year"])
out_flows = inD_grouped.agg(d).reset_index()

#Calculate in_flows
d = {'TotalSeats':'sum'}
inD_grouped = inD.groupby(['Country2', "Year"])
in_flows = inD_grouped.agg(d)

in_flows = in_flows['TotalSeats'].unstack().reset_index()

In [ ]:
def get_data(x):
    x = [y for y in x[2:] if not np.isnan(y)]
    return(x[-1])
in_flows['CURRENT'] = in_flows.apply(get_data, axis=1)
in_flows.shape

# Calculate airport and port flows

In [ ]:
port_data = "/home/public/Data/GLOBAL/INFRA/PORTS/Port_flow_data_Q22020.csv"
port_locations = "/home/public/Data/GLOBAL/INFRA/PORTS/attributed_ports.shp"
airport_data = "/home/wb411133/data/Projects/INFRA/FLOWS/Airport_Volume.csv"

inP = pd.read_csv(port_data)
inP_loc = gpd.read_file(port_locations)
inA = pd.read_csv(airport_data)
# For the simplest version, we are only calculating flows for 2017
inA = inA.loc[inA['Year'] == 2019]

In [ ]:
agg = {'Name':'first','NAme':'first','Country1':'first','Country2':'first','TotalSeats':'sum',"Country Name":"first","Airport1Latitude":'first', "Airport1Longitude":'first',"Airport2Latitude":'first', "Airport2Longitude":'first'}
inA_g = inA.groupby(['Orig','Dest'])
inA_g = inA_g.agg(agg).reset_index()
geoms = inA_g.apply(lambda x: LineString([Point(x['Airport1Longitude'], x['Airport1Latitude']),
                                         Point(x['Airport2Longitude'], x['Airport2Latitude'])]), axis=1)
inA_g = gpd.GeoDataFrame(inA_g, geometry=geoms, crs={'init':'epsg:4326'})


In [ ]:
inA_g.head()

In [ ]:
inA_g.to_file("/home/wb411133/temp/airport_flows.shp")

In [ ]:
# attribute port flows
agg = {'Quarterly deployed capacity (TEU)':'sum'}
inP_g = inP.groupby(["Port1","Port2"])
inP_flows = inP_g.agg(agg).reset_index()

In [ ]:
inP_geoms = inP_flows.merge(inP_loc.loc[:,['LOCODE','geometry']], left_on="Port1", right_on="LOCODE")
inP_geoms.columns = ['Port1','Port2','FLOWS','LOCODE_1','PT_1']
inP_geoms = inP_geoms.merge(inP_loc.loc[:,['LOCODE','geometry']], left_on="Port2", right_on="LOCODE")
inP_geoms.columns = ['Port1','Port2','FLOWS','LOCODE_1','PT_1','LOCODE_2','PT_2']
inP_geoms.drop(['LOCODE_1',"LOCODE_2"], axis=1, inplace=True)
inP_geoms.head()

In [ ]:
sindex = inB.sindex

In [ ]:
inB.loc[list(sindex.nearest([inP_geoms['PT_1'].iloc[0].x, inP_geoms['PT_1'].iloc[0].y]))[0]]['ISO3']

In [ ]:
# attribute port flows with origin and destination country
#inP_geoms['Country1'] = inP_geoms['PT_1'].apply(lambda x: inB.loc[list(sindex.nearest([x.x, x.y]))[0]]['ISO3'])
#inP_geoms['Country2'] = inP_geoms['PT_2'].apply(lambda x: inB.loc[list(sindex.nearest([x.x, x.y]))[0]]['ISO3'])
inP_geoms['Country1'] = inP_geoms['Port1'].apply(lambda x: x[:2])
inP_geoms['Country2'] = inP_geoms['Port2'].apply(lambda x: x[:2])
inP_geoms.head()

In [ ]:
flow_geoms = inP_geoms.apply(lambda x: LineString([x['PT_1'], x['PT_2']]), axis=1)
inP_geoms = gpd.GeoDataFrame(inP_geoms, geometry=flow_geoms, crs={'init':'epsg:4326'})
inP_geoms.drop(["PT_1","PT_2"], axis=1).to_file("/home/wb411133/temp/port_flows.shp")